# GPT Model w/ Descriptions
Combined_Songs_Artists.csv

Model trained on Playlist_Names, Playlist_Descriptions, Songs and Artists combined in one string

In [7]:
import pandas as pd
from datasets import Dataset
import ast
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import os
import csv
import optuna
import torch

In [2]:
os.chdir('..')
os.getcwd()

'/sfs/gpfs/tardis/home/nuf8ms/Documents/MSDS/LLM'

In [2]:
#df2 = pd.read_csv('DS6051-Project/Playlist_data_with_lyrics.csv')
#df3 = pd.read_csv('DS6051-Project/playlist_data.csv')
df = pd.read_csv('../Additional Data/Combined_Songs_Artists.csv')

In [3]:
# Load and parse the dataset

# Convert stringified lists to actual lists
# for col in ['Playlist_Songs', 'Playlist_Artists']:
#     df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

df["Playlist_Songs"] = df["Playlist_Songs"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

## Training Targets 1st Iteration, 3 epoch Loss ~ 3, 5 epoch Loss ~ 2.5
## playlist_data.csv
# Create prompt + output text for GPT-2 training
# def format_example(row):
#     song_lines = [f"{s} - {a}" for s, a in zip(row['Playlist_Songs'], row['Playlist_Artists'])]
#     return f"### Prompt: {row['Playlist_Name']}\n### Playlist:\n" + "\n".join(song_lines)

## Training Targets 2nd Iteration, 5 epoch Loss ~ 1.3
## playlist_data.csv
# def format_example(row):
#     lines = [f"[SONG] {s} [ARTIST] {a}" for s, a in zip(row["Playlist_Songs"], row["Playlist_Artists"])]
#     return f"### Prompt: {row['Playlist_Name']}\n### Playlist:\n" + "\n".join(lines)

## Training targets 3rd Iteration, 5 epoch Loss ~ 1.35
## playlist_data.csv
# def format_example(row):
#     lines = [f"[SONG] {song} [ARTIST] {artist}" for song, artist in zip(row["Playlist_Songs"], row["Playlist_Artists"])]
#     playlist_body = "\n".join(lines)
    
#     # Include Playlist_Description in the prompt for training only
#     return (
#         f"### Prompt: {row['Playlist_Name']}\n"
#         f"### Description: {row['Playlist_Description']}\n"
#         f"### Playlist:\n{playlist_body}"
#     )

## Training targets 4th Iteration, 
## Combined_Songs_Artists.csv
def format_example(row):
    # Use the song-artist strings as-is
    playlist_body = "\n".join(row["Playlist_Songs"])
    return (
        f"### Prompt: {row['Playlist_Name']}\n"
        f"### Description: {row['Playlist_Description']}\n"
        f"### Playlist:\n{playlist_body}"
    )

df['text'] = df.apply(format_example, axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({"text": df['text'].tolist()})

In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token

model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
model.resize_token_embeddings(len(tokenizer))  # In case we add special tokens

# Tokenize
def tokenize(batch):
    encodings = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    encodings["labels"] = encodings["input_ids"].copy()  # 🔥 Add this line
    return encodings
    
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map: 100%|██████████| 2289/2289 [00:07<00:00, 310.51 examples/s]


### Test Script

In [20]:
# Create train/eval split
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

def model_init():
    return GPT2LMHeadModel.from_pretrained("gpt2-medium")

# Define a function for hyperparameter search
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 5e-5),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 3, 20),
        "warmup_steps": trial.suggest_int("warmup_steps", 50, 200),
        "weight_decay": trial.suggest_loguniform("weight_decay", 1e-4, 0.1),
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4])
    }

training_args = TrainingArguments(
    output_dir="./models/hp_search",
    overwrite_output_dir = True,
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    fp16=True if torch.cuda.is_available() else False,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
)

# Initialize Trainer without hardcoding hyperparameters, as they are provided during search
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Run the hyperparameter search with Optuna
best_run = trainer.hyperparameter_search(
    direction="minimize",
    hp_space=hp_space,
    n_trials=10  # Number of trials to run; increase for thorough search
)

print("Best hyperparameters found:", best_run.hyperparameters)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
[I 2025-04-14 19:29:57,720] A new study created in memory with name: no-name-e8216f6d-2678-4564-943b-64c3c7369c6a
/tmp/ipykernel_358383/1843834621.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 5e-5),
/tmp/ipykernel_358383/1843834621.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "weight_decay": trial.suggest_loguniform("weight_decay", 1e-4, 0.1),
Detected kernel version 4.18.0, which is below the recommend

Step,Training Loss,Validation Loss
100,3.524200,3.092731
200,2.820100,2.711983
300,2.779000,2.649030
400,2.775800,2.613460
500,2.668600,2.591468
600,2.664300,2.574363
700,2.661800,2.557748
800,2.520100,2.545507
900,2.623000,2.534736
1000,2.638500,2.523269


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
[I 2025-04-14 19:41:09,729] Trial 0 finished with value: 2.434735059738159 and parameters: {'learning_rate': 6.2777217222155105e-06, 'per_device_train_batch_size': 8, 'num_train_epochs': 13, 'warmup_steps': 151, 'weight_decay': 0.020161829261015833, 'gradient_accumulation_steps': 1}. Best is trial 0 with value: 2.434735059738159.
/tmp/ipykernel_358383/1843834621.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 5e-5),
/tmp/ipykernel_358383/1843834621.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "weight_decay": trial.sugg

Step,Training Loss,Validation Loss
100,3.936300,3.678468
200,3.694900,3.392276
300,3.241900,3.138432
400,3.297900,3.009741
500,3.155700,2.920978
600,3.165000,2.849820
700,2.875700,2.789839
800,2.724000,2.749564
900,2.868700,2.730555
1000,2.883100,2.719564


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
[I 2025-04-14 20:14:28,877] Trial 1 finished with value: 2.4728024005889893 and parameters: {'learning_rate': 1.4576065605822118e-06, 'per_device_train_batch_size': 2, 'num_train_epochs': 19, 'warmup_steps': 96, 'weight_decay': 0.06482198720505387, 'gradient_accumulation_steps': 1}. Best is trial 0 with value: 2.434735059738159.
/tmp/ipykernel_358383/1843834621.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 5e-5),
/tmp/ipykernel_358383/1843834621.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "weight_decay": trial.sugge

...

Step,Training Loss,Validation Loss
100,3.069100,2.738971
200,2.761600,2.599806
300,2.660200,2.552391
400,2.574700,2.516480
500,2.595200,2.490393
600,2.527300,2.470569
700,2.490400,2.455985
800,2.486900,2.443063
900,2.465900,2.428353
1000,2.436100,2.419094


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
[I 2025-04-14 21:30:46,964] Trial 8 finished with value: 2.364872932434082 and parameters: {'learning_rate': 1.122961376201031e-05, 'per_device_train_batch_size': 4, 'num_train_epochs': 17, 'warmup_steps': 107, 'weight_decay': 0.09579287629931685, 'gradient_accumulation_steps': 4}. Best is trial 5 with value: 2.2394657135009766.
/tmp/ipykernel_358383/1843834621.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 5e-5),
/tmp/ipykernel_358383/1843834621.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "weight_decay": trial.sugge

Step,Training Loss,Validation Loss
100,2.901700,2.640264
200,2.572000,2.484294
300,2.482400,2.422452
400,2.406200,2.387951
500,2.351100,2.369979


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
[I 2025-04-14 21:37:22,245] Trial 9 finished with value: 2.3699793815612793 and parameters: {'learning_rate': 2.881730484419015e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 9, 'warmup_steps': 146, 'weight_decay': 0.05501524719495783, 'gradient_accumulation_steps': 4}. Best is trial 5 with value: 2.2394657135009766.


Best hyperparameters found: {'learning_rate': 4.5550259680115846e-05, 'per_device_train_batch_size': 2, 'num_train_epochs': 12, 'warmup_steps': 155, 'weight_decay': 0.00020452071265917842, 'gradient_accumulation_steps': 1}


### Test Script

In [11]:
training_args = TrainingArguments(
    output_dir="./models/gpt2_Combined_Song_Artists_Checkpoints",
    overwrite_output_dir=True,
    num_train_epochs=30,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    fp16=True if torch.cuda.is_available() else False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.371100
100,2.917400
150,2.865700
200,2.789100
250,2.698200
300,2.682200
350,2.684800
400,2.548800
450,2.555700
500,2.565100


TrainOutput(global_step=34350, training_loss=0.9339163249097711, metrics={'train_runtime': 5592.7868, 'train_samples_per_second': 12.278, 'train_steps_per_second': 6.142, 'total_flos': 6.377387669323776e+16, 'train_loss': 0.9339163249097711, 'epoch': 30.0})

In [12]:
model.save_pretrained("./models/gpt2_Combined_Song_Artists")
tokenizer.save_pretrained("./models/gpt2_Combined_Song_Artists")

('./models/gpt2_Combined_Song_Artists/tokenizer_config.json',
 './models/gpt2_Combined_Song_Artists/special_tokens_map.json',
 './models/gpt2_Combined_Song_Artists/vocab.json',
 './models/gpt2_Combined_Song_Artists/merges.txt',
 './models/gpt2_Combined_Song_Artists/added_tokens.json')

In [13]:
def generate_playlist(prompt, max_length=200):
    input_text = f"### Prompt: {prompt}\n### Playlist:\n"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output = model.generate(
        input_ids = input_ids.to('cuda'),
        max_length=max_length,
        temperature=0.9,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result.split("### Playlist:\n")[1].strip()

# Try it out!
prompt = "Road Trip with Classic Rock"
print(generate_playlist(prompt))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Light My Fire : INXS
Rocket Man (I Think It's Going To Be A Long Long Time) : Elton John
Black or White - Single Version : Michael Jackson
Moby Dick : Led Zeppelin
Carry on Wayward Son : Kansas
Hotel California - 2013 Remaster : Eagles
Dream On : Aerosmith
Layla : Derek & The Dominos
The Logical Song - Remastered 2012 : Neil Young
Johnny B. Goode - Single Version : Chuck Berry
More Than a Feeling : Boston
Vultaricious : War
Piano Man : Billy Joel
You've Got a Friend : James Taylor
Moby Dick : Led Zeppelin
Run to Me : Bryan Adams
Carry on Wayward Son : Kansas
Go Your Own Way - 2004 Remaster : Fleetwood Mac
Rock and Roll - Remaster : Led Zeppelin
Learning To Fly :
